In [127]:
import pandas as pd
import numpy as np

import requests

import os
import time
import sys
import yaml

from data_loader import *
from utils import *


pd.set_option("display.max_column",None)
print(os.getcwd())


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


def auto_reload():
    %load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open("config.yaml",'r') as f:
    config = yaml.safe_load(f)


home_dir = config['HOME_DIRECTORY']
home_dir

'/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG'

# API Football:
https://www.api-football.com

![PYTHON LOGO](https://www.api-football.com/public/img/news/archi-beta.jpg)

In [3]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']]

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil
...,...,...,...
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World
1130,1133,Goiano - 3,Brazil
1131,1001,CONCACAF Women U20,World


In [ ]:
find_leagues(leagues_dat,league_name=None,country="France")

# Leagues subset:

In [ ]:
# Configs
major_leagues = ["Premier League","La Liga","Serie A","Bundesliga","Eredivisie","Ligue 1"]
major_countries = ["England","Spain","Italy","Germany","Netherlands","France","Brazil"]
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = [2022,2021,2023,2024] # seasons to pull players and teams stats of



leagues_subset = leagues_dat[leagues_dat.league_name.isin(major_leagues) & leagues_dat.country_name.isin(major_countries)] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [5]:
leagues_subset

,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
2,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,"[{'coverage': {'fixtures': {'events': True, 'l...",71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,https://media.api-sports.io/flags/br.svg
5,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
6,"[{'coverage': {'fixtures': {'events': True, 'l...",78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,https://media.api-sports.io/flags/de.svg
7,"[{'coverage': {'fixtures': {'events': True, 'l...",135,Serie A,League,https://media.api-sports.io/football/leagues/1...,Italy,IT,https://media.api-sports.io/flags/it.svg
8,"[{'coverage': {'fixtures': {'events': True, 'l...",88,Eredivisie,League,https://media.api-sports.io/football/leagues/8...,Netherlands,NL,https://media.api-sports.io/flags/nl.svg
10,"[{'coverage': {'fixtures': {'events': True, 'l...",140,La Liga,League,https://media.api-sports.io/football/leagues/1...,Spain,ES,https://media.api-sports.io/flags/es.svg


In [18]:
liverpool_fixtures = get_team_fixtures("liverpool",2024,39)

In [25]:
home_fixtures = list(liverpool_fixtures[liverpool_fixtures.teams_home_name == "Liverpool"]['fixture_id'])
away_fixtures = list(liverpool_fixtures[liverpool_fixtures.teams_away_name == "Liverpool"]['fixture_id'])

In [26]:
home_fixtures + away_fixtures

[1208036,
 1208056,
 1208066,
 1208094,
 1208116,
 1208127,
 1208147,
 1208177,
 1208197,
 1208218,
 1208248,
 1208268,
 1208291,
 1208297,
 1208322,
 1208337,
 1208357,
 1208376,
 1208396,
 1208022,
 1208047,
 1208080,
 1208086,
 1208103,
 1208142,
 1208161,
 1208191,
 1208212,
 1208228,
 1208234,
 1208253,
 1208166,
 1208305,
 1208280,
 1208327,
 1208349,
 1208367,
 1208386]

In [30]:
player_stat_url = "https://v3.football.api-sports.io/fixtures/players?fixture={}".format(1208047)
fixture_dat = requests.get(player_stat_url,headers=headers_api_sport)

In [32]:
sample_fix = pd.json_normalize(fixture_dat.json()['response'])['players']

In [100]:
auto_reload()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
team_fixtures = get_team_fixtures(team = "Liverpool",season = 2022)

Pulling for Liverpool,2022 with league_id: 39


In [62]:
team_fixtures['winner'] = np.where(team_fixtures.teams_home_winner == True,team_fixtures.teams_home_name,
                                   np.where(team_fixtures.teams_away_name == True,team_fixtures.teams_away_name,'Draw'))

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_45801/1557615184.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  team_fixtures['score_penalty_away'].fillna(0)[1]


np.int64(0)

In [67]:
team_fixtures[['teams_home_name','teams_away_name','goals_home','goals_away','winner']]

,teams_home_name,teams_away_name,goals_home,goals_away,winner
1,Fulham,Liverpool,2,2,Draw
19,Liverpool,Crystal Palace,1,1,Draw
29,Manchester United,Liverpool,2,1,Manchester United
31,Liverpool,Bournemouth,9,0,Liverpool
48,Liverpool,Newcastle,2,1,Liverpool
50,Everton,Liverpool,0,0,Draw
70,Liverpool,Brighton,3,3,Draw
84,Arsenal,Liverpool,3,2,Arsenal
96,Liverpool,Manchester City,1,0,Liverpool
101,Liverpool,West Ham,1,0,Liverpool


In [88]:
fixtures = team_fixtures

In [89]:
fixtures.head()

,fixture_id,fixture_referee,fixture_timezone,fixture_date,fixture_timestamp,fixture_periods_first,fixture_periods_second,fixture_venue_id,fixture_venue_name,fixture_venue_city,fixture_status_long,fixture_status_short,fixture_status_elapsed,fixture_status_extra,league_id,league_name,league_country,league_logo,league_flag,league_season,league_round,league_standings,teams_home_id,teams_home_name,teams_home_logo,teams_home_winner,teams_away_id,teams_away_name,teams_away_logo,teams_away_winner,goals_home,goals_away,score_halftime_home,score_halftime_away,score_fulltime_home,score_fulltime_away,score_extratime_home,score_extratime_away,score_penalty_home,score_penalty_away,winner
1,867947,A. Madley,UTC,2022-08-06T11:30:00+00:00,1659785400,1659785400,1659789000,535,Craven Cottage,London,Match Finished,FT,90,None,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2022,Regular Season - 1,True,36,Fulham,https://media.api-sports.io/football/teams/36.png,None,40,Liverpool,https://media.api-sports.io/football/teams/40.png,None,2,2,1,0,2,2,None,None,None,None,Draw
19,867961,P. Tierney,UTC,2022-08-15T19:00:00+00:00,1660590000,1660590000,1660593600,550,Anfield,Liverpool,Match Finished,FT,90,None,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2022,Regular Season - 2,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,None,52,Crystal Palace,https://media.api-sports.io/football/teams/52.png,None,1,1,0,1,1,1,None,None,None,None,Draw
29,867972,M. Oliver,UTC,2022-08-22T19:00:00+00:00,1661194800,1661194800,1661198400,556,Old Trafford,Manchester,Match Finished,FT,90,None,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2022,Regular Season - 3,True,33,Manchester United,https://media.api-sports.io/football/teams/33.png,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,False,2,1,1,0,2,1,None,None,None,None,Manchester United
31,867981,S. Attwell,UTC,2022-08-27T14:00:00+00:00,1661608800,1661608800,1661612400,550,Anfield,Liverpool,Match Finished,FT,90,None,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2022,Regular Season - 4,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,False,9,0,5,0,9,0,None,None,None,None,Liverpool
48,867994,A. Marriner,UTC,2022-08-31T19:00:00+00:00,1661972400,1661972400,1661976000,550,Anfield,Liverpool,Match Finished,FT,90,None,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg,2022,Regular Season - 5,True,40,Liverpool,https://media.api-sports.io/football/teams/40.png,True,34,Newcastle,https://media.api-sports.io/football/teams/34.png,False,2,1,0,1,2,1,None,None,None,None,Liverpool


In [113]:
liverpool_dat = read_fixtures_for_season("Liverpool",2024)

Pulling for Liverpool,2024 with league_id: 39


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:104: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

In [118]:
liverpool_dat['win'] = (liverpool_dat.team == liverpool_dat.outcome).astype("int")

In [119]:
liverpool_dat.head()

,player.id,player.name,offsides,games.minutes,games.number,games.position,games.rating,games.captain,games.substitute,shots.total,shots.on,goals.total,goals.conceded,goals.assists,goals.saves,passes.total,passes.key,passes.accuracy,tackles.total,tackles.blocks,tackles.interceptions,duels.total,duels.won,dribbles.attempts,dribbles.success,dribbles.past,fouls.drawn,fouls.committed,cards.yellow,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,outcome,team,win
0,280,Alisson,NaN,90.0,1,G,7.3,False,False,NaN,NaN,NaN,0,0.0,2.0,35.0,NaN,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0.0,1208036,2,2,2,1,0,0,Brighton,Liverpool,0
1,283,Trent Alexander-Arnold,NaN,72.0,66,D,7.9,False,False,1.0,NaN,NaN,0,0.0,NaN,50.0,4.0,41,2.0,1.0,NaN,4.0,3.0,1.0,1.0,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208036,2,2,2,1,0,0,Brighton,Liverpool,0
2,1145,Ibrahima Konaté,NaN,90.0,5,D,7.9,False,False,2.0,1.0,NaN,0,0.0,NaN,76.0,NaN,73,3.0,1.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208036,2,2,2,1,0,0,Brighton,Liverpool,0
3,290,Virgil van Dijk,NaN,90.0,4,D,7.2,True,False,1.0,1.0,NaN,0,0.0,NaN,62.0,NaN,59,1.0,NaN,1.0,7.0,4.0,NaN,NaN,1.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208036,2,2,2,1,0,0,Brighton,Liverpool,0
4,289,Andrew Robertson,NaN,90.0,26,D,7.7,False,False,2.0,2.0,NaN,0,0.0,NaN,67.0,3.0,60,1.0,NaN,NaN,3.0,2.0,1.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208036,2,2,2,1,0,0,Brighton,Liverpool,0


In [125]:
auto_reload()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
find_team("Eintr")

,team_name,league
10,Eintracht Frankfurt,78


In [ ]:
# Lets see how Hugo Ekitike compares:
eintract_dat = read_fixtures_for_season("Eintracht Frankfurt",2025)

Pulling for Eintracht Frankfurt,2024 with league_id: 78


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:104: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

In [131]:
eintract_dat = lower_columns(eintract_dat)

In [141]:
ekitike_dat = eintract_dat[eintract_dat.player_name == 'Hugo Ekitiké']
ekitike_dat.head()

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,outcome,team,win
9,174565,Hugo Ekitiké,NaN,65.0,11,F,8.3,False,False,2.0,1.0,1.0,0,1.0,NaN,26.0,5.0,24,1.0,NaN,1.0,15.0,5.0,7.0,2.0,NaN,NaN,1.0,0,0,NaN,NaN,0,0,NaN,1223987,3,3,3,2,1,0,Draw,Eintracht Frankfurt,draw
9,174565,Hugo Ekitiké,NaN,78.0,11,F,6.3,False,False,3.0,1.0,NaN,0,0.0,NaN,18.0,1.0,12,1.0,NaN,1.0,12.0,6.0,6.0,2.0,NaN,2.0,NaN,0,0,NaN,NaN,0,0,NaN,1224004,2,2,2,1,0,0,Draw,Eintracht Frankfurt,draw
9,174565,Hugo Ekitiké,2.0,66.0,11,F,7.2,False,False,3.0,2.0,1.0,0,0.0,NaN,15.0,NaN,11,NaN,NaN,1.0,8.0,4.0,5.0,2.0,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1224022,3,3,3,2,3,2,Draw,Eintracht Frankfurt,draw
9,174565,Hugo Ekitiké,NaN,90.0,11,F,8,False,False,2.0,2.0,2.0,0,0.0,NaN,22.0,NaN,15,NaN,NaN,NaN,9.0,2.0,3.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1224050,7,7,7,4,2,1,Draw,Eintracht Frankfurt,draw
9,174565,Hugo Ekitiké,1.0,89.0,11,F,7.7,False,False,1.0,1.0,NaN,0,1.0,NaN,23.0,3.0,19,NaN,NaN,NaN,10.0,6.0,3.0,3.0,NaN,1.0,1.0,0,0,NaN,NaN,0,0,NaN,1224069,1,1,1,1,0,0,Draw,Eintracht Frankfurt,draw


In [172]:
liverpool_dat.games_position.value_counts()

games_position
M    292
D    261
F    127
G     80
Name: count, dtype: int64

In [ ]:
ekitike_dat['dribble_success_rate'] = (ekitike_dat.dribbles_success/ekitike_dat.dribbles_attempts) * 100
ekitike_dat['target_shot_cr'] = (ekitike_dat.goals_total/ekitike_dat.shots_on) * 100


/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_45801/1383071472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ekitike_dat['dribble_success_rate'] = (ekitike_dat.dribbles_success/ekitike_dat.dribbles_attempts) * 100
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_45801/1383071472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ekitike_dat['target_shot_cr'] = (ekitike_dat.goals_total/ekitike_dat.shots_on) * 100


In [186]:
ekitike_dat

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,outcome,team,win,dribble_success_rate,target_shot_cr
9,174565,Hugo Ekitiké,NaN,65.0,11,F,8.3,False,False,2.0,1.0,1.0,0,1.0,NaN,26.0,5.0,24,1.0,NaN,1.0,15.0,5.0,7.0,2.0,NaN,NaN,1.0,0,0,NaN,NaN,0,0,NaN,1223987,3,3,3,2,1,0,Draw,Eintracht Frankfurt,draw,28.571429,100.000000
9,174565,Hugo Ekitiké,NaN,78.0,11,F,6.3,False,False,3.0,1.0,NaN,0,0.0,NaN,18.0,1.0,12,1.0,NaN,1.0,12.0,6.0,6.0,2.0,NaN,2.0,NaN,0,0,NaN,NaN,0,0,NaN,1224004,2,2,2,1,0,0,Draw,Eintracht Frankfurt,draw,33.333333,NaN
9,174565,Hugo Ekitiké,2.0,66.0,11,F,7.2,False,False,3.0,2.0,1.0,0,0.0,NaN,15.0,NaN,11,NaN,NaN,1.0,8.0,4.0,5.0,2.0,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1224022,3,3,3,2,3,2,Draw,Eintracht Frankfurt,draw,40.000000,50.000000
9,174565,Hugo Ekitiké,NaN,90.0,11,F,8,False,False,2.0,2.0,2.0,0,0.0,NaN,22.0,NaN,15,NaN,NaN,NaN,9.0,2.0,3.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1224050,7,7,7,4,2,1,Draw,Eintracht Frankfurt,draw,NaN,100.000000
9,174565,Hugo Ekitiké,1.0,89.0,11,F,7.7,False,False,1.0,1.0,NaN,0,1.0,NaN,23.0,3.0,19,NaN,NaN,NaN,10.0,6.0,3.0,3.0,NaN,1.0,1.0,0,0,NaN,NaN,0,0,NaN,1224069,1,1,1,1,0,0,Draw,Eintracht Frankfurt,draw,100.000000,NaN
10,174565,Hugo Ekitiké,NaN,89.0,11,F,7.9,False,False,1.0,1.0,1.0,0,0.0,NaN,25.0,3.0,17,3.0,NaN,1.0,10.0,5.0,3.0,2.0,NaN,NaN,1.0,0,0,NaN,NaN,0,0,NaN,1224086,2,2,2,0,2,0,Draw,Eintracht Frankfurt,draw,66.666667,100.000000
9,174565,Hugo Ekitiké,NaN,61.0,11,F,6.9,False,False,2.0,1.0,NaN,0,0.0,NaN,13.0,NaN,13,NaN,NaN,NaN,8.0,4.0,3.0,2.0,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1224104,1,1,1,0,3,2,Draw,Eintracht Frankfurt,draw,66.666667,NaN
9,174565,Hugo Ekitiké,NaN,81.0,11,F,7.3,False,False,4.0,1.0,1.0,0,0.0,NaN,23.0,NaN,23,1.0,NaN,NaN,10.0,3.0,4.0,1.0,NaN,NaN,1.0,0,0,NaN,NaN,0,0,NaN,1224122,4,4,4,1,1,1,Draw,Eintracht Frankfurt,draw,25.000000,100.000000
9,174565,Hugo Ekitiké,NaN,89.0,11,F,7.7,False,False,5.0,3.0,1.0,0,0.0,NaN,14.0,1.0,9,NaN,NaN,NaN,11.0,1.0,3.0,1.0,NaN,NaN,4.0,0,0,NaN,NaN,0,0,NaN,1224131,2,2,2,1,0,0,Draw,Eintracht Frankfurt,draw,33.333333,33.333333
10,174565,Hugo Ekitiké,NaN,90.0,11,F,6.9,False,False,2.0,1.0,NaN,0,0.0,NaN,20.0,2.0,16,NaN,NaN,NaN,7.0,NaN,2.0,NaN,NaN,NaN,2.0,0,0,NaN,NaN,0,0,NaN,1224149,1,1,1,0,1,0,Draw,Eintracht Frankfurt,draw,NaN,NaN


In [ ]:
liverpool_dat['dribble_success_rate'] = (liverpool_dat.dribbles_success/liverpool_dat.dribbles_attempts) * 100
liverpool_dat['target_shot_cr'] = (liverpool_dat.goals_total/liverpool_dat.shots_on) * 100

In [ ]:
# quick knn clustering
from sklearn.cluster import KMeans

# who is ekitike most similar to 
dribble_comparison_dat = pd.concat([liverpool_dat[['games_position','player_name','dribble_success_rate','target_shot_cr']].fillna(0),ekitike_dat[['games_position','player_name','dribble_success_rate','target_shot_cr']].fillna(0) ],axis = 0)
avg_dribble_success = dribble_comparison_dat[dribble_comparison_dat.games_position.isin(['F','M'])].groupby(['player_name']).agg(n_games = ('player_name','size'),
                                                                          avg_dribble_success = ('dribble_success_rate','mean'),
                                                                          std_dribble_success = ('dribble_success_rate','std'),
                                                                          avg_target_shot_cr = ('target_shot_cr','mean'),
                                                                          std_target_shot_cr = ('target_shot_cr','std'),
                                                                          )

km = KMeans(6,random_state= 5)
avg_dribble_success['cluster_dribble'] = km.fit_predict(avg_dribble_success[['avg_dribble_success']])

km = KMeans(6,random_state= 5)
avg_dribble_success['cluster_shot_cr'] = km.fit_predict(avg_dribble_success[['avg_target_shot_cr']])


avg_dribble_success.sort_values(['avg_dribble_success'],ascending=[0,0])


,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,cluster_dribble,cluster_shot_cr
player_name,,,,,,,
Cody Gakpo,35,49.047619,44.591988,20.952381,38.633280,1,4
Ryan Gravenberch,37,46.589447,41.327724,0.000000,0.000000,1,3
Hugo Ekitiké,33,43.953824,29.855801,27.272727,39.046398,1,1
Mohamed Salah,38,43.027455,31.231097,46.710526,42.338701,4,2
Luis Díaz,38,40.526316,39.030043,18.157895,33.519277,4,4
Dominik Szoboszlai,36,36.342593,43.831965,12.500000,30.178043,4,5
Curtis Jones,32,21.354167,37.682501,6.250000,21.060588,3,0
Alexis Mac Allister,35,21.122449,34.627901,12.857143,32.860796,3,5
Diogo Jota,29,19.540230,34.523102,16.666667,35.634832,3,4


In [193]:
avg_dribble_success.sort_values(['avg_target_shot_cr'],ascending=[0])

,n_games,avg_dribble_success,std_dribble_success,avg_target_shot_cr,std_target_shot_cr,cluster_dribble,cluster_shot_cr
player_name,,,,,,,
Mohamed Salah,38,43.027455,31.231097,46.710526,42.338701,4,2
Hugo Ekitiké,33,43.953824,29.855801,27.272727,39.046398,1,1
Cody Gakpo,35,49.047619,44.591988,20.952381,38.633280,1,4
Luis Díaz,38,40.526316,39.030043,18.157895,33.519277,4,4
Diogo Jota,29,19.540230,34.523102,16.666667,35.634832,3,4
Alexis Mac Allister,35,21.122449,34.627901,12.857143,32.860796,3,5
Dominik Szoboszlai,36,36.342593,43.831965,12.500000,30.178043,4,5
Darwin Núñez,35,13.333333,31.258986,6.666667,20.690688,0,0
Curtis Jones,32,21.354167,37.682501,6.250000,21.060588,3,0
